In [ ]:
query = """
SELECT table_name, column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'TEST'
ORDER BY table_name, ordinal_position
"""

schema_df = spark.read.format("snowflake").options(**sfOptions).option("query", query).load()
schema_df.show()

+----------+-----------+---------+
|TABLE_NAME|COLUMN_NAME|DATA_TYPE|
+----------+-----------+---------+
|     USERS|    USER_ID|   NUMBER|
|     USERS|   USERNAME|     TEXT|
|     USERS|       ROLE|     TEXT|
|     USERS|      EMAIL|     TEXT|
|     USERS|   PASSWORD|     TEXT|
+----------+-----------+---------+



In [ ]:
schema_data = schema_df.collect()
print(schema_data)

[Row(TABLE_NAME='USERS', COLUMN_NAME='USER_ID', DATA_TYPE='NUMBER'), Row(TABLE_NAME='USERS', COLUMN_NAME='USERNAME', DATA_TYPE='TEXT'), Row(TABLE_NAME='USERS', COLUMN_NAME='EMAIL', DATA_TYPE='TEXT'), Row(TABLE_NAME='USERS', COLUMN_NAME='PASSWORD', DATA_TYPE='TEXT'), Row(TABLE_NAME='USERS', COLUMN_NAME='ROLE', DATA_TYPE='TEXT')]


In [ ]:
# Collect schema data as a Python list of dictionaries

table_schemas = {}
for row in schema_data:
    table_name = row["table_name"]
    if table_name not in table_schemas:
        table_schemas[table_name] = []
    table_schemas[table_name].append({"column_name": row["column_name"], "data_type": row["data_type"]})


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File /databricks/spark/python/pyspark/sql/types.py:2031, in Row.__getitem__(self, item)
   2028 try:
   2029     # it will be slow when it has many fields,
   2030     # but this will not be used in normal cases
-> 2031     idx = self.__fields__.index(item)
   2032     return super(Row, self).__getitem__(idx)

ValueError: 'table_name' is not in list

During handling of the above exception, another exception occurred:

ValueError                                Traceback (most recent call last)
File <command-939895357424119>:5
      3 table_schemas = {}
      4 for row in schema_data:
----> 5     table_name = row["table_name"]
      6     if table_name not in table_schemas:
      7         table_schemas[table_name] = []

File /databricks/spark/python/pyspark/sql/types.py:2036, in Row.__getitem__(self, item)
   2034     raise KeyError(item

In [ ]:
# ! pip install snowflake-connector-python

In [1]:
import snowflake.connector
import os

# Set your Snowflake credentials


# Establish the connection
conn = snowflake.connector.connect(
    user=USER,
    password=PASSWORD,
    account=ACCOUNT,
    database=DATABASE,
    schema=SCHEMA
)

# Create a cursor object
cursor = conn.cursor()

try:
    # Query to retrieve all tables in the specified schema
    query = f"""
    SELECT table_name,
           created AS create_date,
           last_altered AS modify_date
    FROM information_schema.tables
    WHERE table_schema = '{SCHEMA}' AND table_type = 'BASE TABLE'
    ORDER BY table_name;
    """
    
    cursor.execute(query)
    
    # Fetch all results
    tables = cursor.fetchall()
    
    # Print the details of each table
    for table in tables:
        print(f"Table Name: {table[0]}, Created: {table[1]}, Last Modified: {table[2]}")

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()


NameError: name 'USER' is not defined